# Imports

In [2]:
import duckdb
import os
import pyarrow
import pandas as pd

# Data

<table>
  <thead>
    <tr>
      <th><strong>#</strong></th>
      <th><strong>컬럼명</strong></th>
      <th><strong>컬럼 설명</strong></th>
      <th><strong>컬럼 타입</strong></th>
      <th><strong>비고</strong></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>enter.main_page</td>
      <td>서비스 메인페이지 진입</td>
      <td></td>
      <td></td>
    </tr>
    <tr>
      <td>2</td>
      <td>enter.signup_page</td>
      <td>회원가입 페이지 진입</td>
      <td></td>
      <td></td>
    </tr>
    <tr>
      <td>3</td>
      <td>complete.signup</td>
      <td>회원가입 완료</td>
      <td>로그인 방식</td>
      <td></td>
    </tr>
    <tr>
      <td>4</td>
      <td>enter.content_page</td>
      <td>콘텐츠 개별 페이지 진입</td>
      <td>콘텐츠 id</td>
      <td></td>
    </tr>
    <tr>
      <td>5</td>
      <td>click.content_page_start_content_button</td>
      <td>콘텐츠 시청하기 버튼 클릭</td>
      <td>콘텐츠 id, 버튼 이름</td>
      <td></td>
    </tr>
    <tr>
      <td>6</td>
      <td>click.content_page_more_review_button</td>
      <td>콘텐츠 후기 더보기 버튼 클릭</td>
      <td>콘텐츠 id</td>
      <td></td>
    </tr>
    <tr>
      <td>7</td>
      <td>enter.payment_page</td>
      <td></td>
      <td></td>
      <td></td>
    </tr>
    <tr>
      <td>8</td>
      <td>complete.subscription</td>
      <td>첫 결제 완료</td>
      <td>요금제 종류, 쿠폰 종류, 쿠폰 혜택 종류, 정상가, 할인가, 할인액, 결제수단 타입</td>
      <td>첫 결제가 중복인 경우는 첫 결제 환불 후 다시 결제한 케이스로 간주</td>
    </tr>
    <tr>
      <td>9</td>
      <td>renew.subscription</td>
      <td>정기 결제 완료</td>
      <td>요금제 종류, 쿠폰 종류, 쿠폰 혜택 종류, 정상가, 할인가, 할인액, 결제수단 타입</td>
      <td></td>
    </tr>
    <tr>
      <td>10</td>
      <td>resubscribe.subscription</td>
      <td>만료 후 재구독 완료</td>
      <td>요금제 종류, 쿠폰 종류, 쿠폰 혜택 종류, 정상가, 할인가, 할인액, 결제수단 타입</td>
      <td></td>
    </tr>
    <tr>
      <td>11</td>
      <td>start.free_trial</td>
      <td></td>
      <td>콘텐츠 id, 요금제 종류, 정상가</td>
      <td></td>
    </tr>
    <tr>
      <td>12</td>
      <td>start.content</td>
      <td>콘텐츠 시청 시작</td>
      <td>콘텐츠 id, 콘텐츠 이용가</td>
      <td>22년 8월부터 수집된 테이블입니다.</td>
    </tr>
    <tr>
      <td>13</td>
      <td>enter.episode_page</td>
      <td>에피소드 시청 시작</td>
      <td>콘텐츠 id, 에피소드 id, 무료체험 여부</td>
      <td>콘텐츠는 여러개의 에피소드로 구성되어있습니다. ex) 콘텐츠 = 빅뱅이론 / 에피소드 = 빅뱅이론 1화, 빅뱅이론 2화 ....</td>
    </tr>
    <tr>
      <td>14</td>
      <td>complete.episode</td>
      <td>에피소드 시청 완료</td>
      <td>콘텐츠 id, 에피소드 id</td>
      <td></td>
    </tr>
    <tr>
      <td>15</td>
      <td>click_episode_page_related_comment_box</td>
      <td>에피소드 페이지 내 댓글보기 버튼 클릭</td>
      <td>콘텐츠 id, 에피소드 id, 댓글 id</td>
      <td></td>
    </tr>
    <tr>
      <td>16</td>
      <td>end.content</td>
      <td>콘텐츠 시청 완료</td>
      <td>콘텐츠 id</td>
      <td>콘텐츠 내 모든 에피소드를 시청하면 해당 테이블에 로그가 쌓임</td>
    </tr>
    <tr>
      <td>17</td>
      <td>click.cancel_plan_button</td>
      <td>구독 취소 버튼 클릭</td>
      <td></td>
      <td>구독 취소 버튼을 클릭했다면 구독 취소가 이루어진 것으로 간주</td>
    </tr>
  </tbody>
</table>

In [4]:
# Define the directory containing the CSV files
data_path = './data/주제 2. 구독서비스 프로덕트 데이터 분석'

In [5]:
# Convert csv files into parquet format
def convert_csv_to_parquet(csv_data_path, parquet_data_path):
    for file_name in os.listdir(csv_data_path):
        # Check if the file is a CSV
        if file_name.endswith('.csv'):
            # Create a table name by stripping the '.csv' and replacing dots with underscores
            file_name_without_extension = file_name.replace('.csv', '').replace('.', '_')
            # Read csv with python and convert file to parquet
            df = pd.read_csv(f'./{csv_data_path}/{file_name}', low_memory=False)
            df.to_parquet(f'./{parquet_data_path}/{file_name_without_extension}.parquet')

convert_csv_to_parquet(csv_data_path=data_path, parquet_data_path='./data/data_parquet')

In [6]:
# Upload all the csv files from a data directory into duckdb
def upload_data_folder_to_duckdb(data_path, db_name):
    # Connect to DuckDB database (will create a new one if it doesn't exist)
    con = duckdb.connect(db_name)
    # Iterate through all parquet files in the folder
    for file_name in os.listdir(data_path):
        # Check if the file is a parquet
        if file_name.endswith('.parquet'):
            # Create a table name by stripping the '.parquet' and replacing dots with underscores
            table_name = file_name.replace('.parquet', '')
            
            # Construct the full path to the parquet file
            parquet_file_path = os.path.join(data_path, file_name)
            
            try:
                # Create a table and load the parquet data into DuckDB
                con.execute(f"""
                CREATE TABLE {table_name} AS
                SELECT * FROM read_parquet('{parquet_file_path}');
                """)
            except duckdb.CatalogException as e:
                # If the table already exists, catch the exception and print a message
                print(f"The table: '{table_name}' already exists.")
    
            except Exception as e:
                # Catch any other exceptions that may occur
                print(f"An error occurred: {e}")
            else:
                # If there were no errors
                print(f"Inserted {file_name} into table {table_name}")

    # Close the connection when done
    con.close()


# Define the directory containing the parquet files
data_path = './data/data_parquet'
# Define the database name
db_name='my_database.duckdb'
upload_data_folder_to_duckdb(data_path=data_path, db_name=db_name)

Inserted click_episode_page_related_comment_box.parquet into table click_episode_page_related_comment_box
Inserted start_content.parquet into table start_content
Inserted click_content_page_more_review_button.parquet into table click_content_page_more_review_button
Inserted enter_payment_page.parquet into table enter_payment_page
Inserted click_cancel_plan_button.parquet into table click_cancel_plan_button
Inserted complete_signup.parquet into table complete_signup
Inserted enter_content_page.parquet into table enter_content_page
Inserted end_content.parquet into table end_content
Inserted enter_main_page.parquet into table enter_main_page
Inserted enter_episode_page.parquet into table enter_episode_page
Inserted enter_signup_page.parquet into table enter_signup_page
Inserted click_content_page_start_content_button.parquet into table click_content_page_start_content_button
Inserted resubscribe_subscription.parquet into table resubscribe_subscription
Inserted renew_subscription.parquet 